<a href="https://colab.research.google.com/github/khaoulaloudiyi/QuickNews/blob/master/Presse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dateparser

     |████████████████████████████████| 368kB 4.8MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wH3TqjVVCuHIa70PukYjt5PGLe_JyNq5wXXKwc7M48RRNgBHxjoct0
Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive//MediaInsights

/content/drive/My Drive/MediaInsights


In [ ]:
!pwd

/content/drive/My Drive/MediaInsights


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from datetime import datetime
import pandas as pd
import csv
import dateparser
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from collections import Counter 
from string import punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stop_words
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
import nltk 
from nltk.corpus import stopwords

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
nltk.download('stopwords')
swa = stopwords.words('french')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Cette partie doit être executer une seule fois pour créer le header des deux fichiers csv

#Ce fichier joue un rôle principal pour la mise à jour puisqu'il contient le champ lastDate qui réfère à la date ou la dernière mise à jour a été effectué et le champ maxid pour faciliter la recherche du plus grand Id 
with open('presse_lastUpdate.csv','w',encoding="utf-8") as csv_file:
    fieldnames=['country', 'Publication', 'maxId', 'lastDate']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
#Ce fichier est réservé pour stocker les différents articles récupéré grâce au scraping 
with open('Articles.csv','w',encoding="utf-8") as csv_file:
    fieldnames=['Id','Title','Author','Date','Publication','URL','Content','Language','Country', "sentiment", "Summary"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()


In [ ]:
#D'abord il faut comprendre la structure des sites web de la presse, chaque presse possède un archive où tous les articles sont stocké, sous forme d'une dizaine d'articles dans plusieurs pages, 
#chaque page de ce dernier contient plusieurs articles référencé par des liens, et tous ces articles possède le même format et contient l'ensemble des informations sauf 
#l'information sur le "sentiment"

#Le principe du Scraping qu'on a utilisé est le suivant "exemple avec Hespress Ar mais la même chose se répète avec les autres presses":
#d'abord on effectue une étude pour comprendre la structure d'une page de l'archive (code html) afin de récupérer les liens des articles seulement
#apres on analyse la structure d'un de ces articles (html) , On parcour l'ensemble des pages de l'archive grâce à la fonction "add_newArticles_hespres" cette
#fonction utilise la variable p qui représente le numéro de page de l'archive a scrapé, on scrape cette page puis on collecte les liens
#href des articles et pour chaque lien on scrape l'article avec la fonction "scrap_article_Hespress" et on l'ajout à la dataset Articles


In [ ]:
#tokenizing sentences into words
def tokenizer(s):
    tokens = []
    for word in s.split(' '):
        tokens.append(word.strip().lower()) #strip remove spaces at the beginning and at the end of the string
    return tokens

#tokenizing document into sentences 
def sent_tokenizer(s):
    sents = []
    for sent in s.split('.'):
        sents.append(sent.strip())
    return sents

def count_words(tokens):
    word_counts = {}
    for token in tokens:
        if token not in stop_words and token not in STOP_WORDS and token not in swa and token not in punctuation:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts

def word_freq_distribution(word_counts):
    freq_dist = {}
    max_freq = max(word_counts.values())
    for word in word_counts.keys():  
        freq_dist[word] = (word_counts[word]/max_freq)
    return freq_dist

def score_sentences(sents, freq_dist, max_len=40):
    sent_scores = {}  
    for sent in sents:
        words = sent.split(' ')
        for word in words:
            if word.lower() in freq_dist.keys():
                if len(words) < max_len:
                    if sent not in sent_scores.keys():
                        sent_scores[sent] = freq_dist[word.lower()]
                    else:
                        sent_scores[sent] += freq_dist[word.lower()]
    return sent_scores

def summarize(sent_scores, k):
    top_sents = Counter(sent_scores) 
    summary = ''
    scores = []
    
    top = top_sents.most_common(k)
    for t in top: 
        summary += t[0].strip()+'. '
        scores.append((t[1], t[0]))
    return summary[:-1], scores

In [ ]:
#le matin
def scrap_article_LeMatin(site, date_max, Id, writer):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)

    title = soup.find("h1", {"id": "title"})
    title = title.text

    time = soup.find("div", {"class": "col-lg-7 col-12 float-left text-left"}).find('time').text
    date_publication = dateparser.parse(time)
    year = date_publication.strftime("%Y")
    month = date_publication.strftime("%m")

    content_a = soup.findAll("div", {"class": "card-body p-2"}) 
    content = " "
    for x in content_a:
        for c in x.findAll('p'):
            if c.text != "\n":
                content +=  c.text + "\n"
    
    
    author ='Le matin' 
    language = 'Fr'
    country = 'Morocco'
    publication = 'LE MATIN'
    
    if(date_publication <= date_max):
        return False, date_publication, Id
    Id = Id + 1
    
    #sentiment analysis
    en_blob = TextBlob(str(content))
    sent = en_blob.sentiment.polarity
    if sent <-0.05 :
        sentiment = "Negative"
    elif sent > -0.05 and sent < 0.05 :
        sentiment = "Neutre"
    else :
        sentiment = "Positive"

    #Summarinzing
    tokens = tokenizer(content)
    sents = sent_tokenizer(content)
    word_counts = count_words(tokens)
    freq_dist = word_freq_distribution(word_counts)
    sent_scores = score_sentences(sents, freq_dist)
    summary, summary_sent_scores = summarize(sent_scores, 3)

    writer.writerow({'Id': Id,'Title': title,'Author': author,'Date':date_publication,'Publication': publication,'URL': site,'Content': content,'Language': language,'Country': country,'sentiment':sentiment, 'Summary':summary})
    return True, date_publication, Id

def add_newArticles_LeMatin(Id, date_max):
    p = 1
    stay = True
    fieldnames=['Id','Title','Author','Date','Publication','URL','Content','Language','Country','sentiment','Summary']
    with open('Articles.csv','a+',encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        lastDate= date_max
        while stay:
            site= "https://lematin.ma/archives/maroc/"+str(p)
            p = p+1
            hdr = {'User-Agent': 'Mozilla/5.0'}
            req = Request(site,headers=hdr)
            page = urlopen(req)
            soup = BeautifulSoup(page)

            links = soup.findAll("a")
            i=0
            websites = []
            for link in links:
                try:
                    res = link["href"].split("/")
                    if res[0] == '':
                        if len(res)>3 and str.isdigit(res[2]):
                            i= i+1
                            stay, date, Id = scrap_article_LeMatin("https://lematin.ma"+link["href"],date_max, Id, writer)
                            if date > lastDate:
                                lastDate = date
                        if (len(res)>1 and res[1] == "archives") or not stay:
                            break
                except KeyError : #or urllib.error.HTTPError:
                    pass
            if i == 0:
                stay = False
    return Id, lastDate

def update_LeMatin():
    fieldnames=['country', 'Publication', 'maxId', 'lastDate']
    with open('presse_lastUpdate.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        readData = [row for row in reader]
        found = False
        id_max = 0 
        lastdate = datetime(1 ,1 ,1 , 0, 0)
        for row in readData:
            if id_max < int(row['maxId']):
                id_max = int(row['maxId'])
            if row['country'] == "Morocco" and row['Publication'] == "LE MATIN" :
                lastdate = pd.to_datetime(row['lastDate'])  
                found = True
       
        id_max, lastdate = add_newArticles_LeMatin(id_max, lastdate)
    if found :
        with open('presse_lastUpdate.csv', "w", newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for row in readData:
                if row['country'] == "Morocco" and row['Publication'] == "LE MATIN" :
                    row['maxId'] = id_max
                    row['lastDate'] = lastdate
                    break
            writer.writerows(readData)
    if not found:
        with open('presse_lastUpdate.csv', "a+", newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow({'country' :'Morocco', 'Publication' : 'LE MATIN',  'maxId' : id_max, 'lastDate' :lastdate})


In [ ]:
def scrap_article_HespressFR(site, date_max, Id, writer):
    #url = "https://www.hespress.com/international/435842.html"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site,headers=hdr)
    try :
        page = urlopen(req)
        soup = BeautifulSoup(page)

        #title = soup.select('h1.page_title')[0].text.strip()
        title = soup.find("div", {"class": "col article-container"}).find('h1').text
        if title.startswith("Températures minimales et maximales") or title.startswith("Prévisions météorologiques pour la journée du ") or title.startswith("Revue de presse quotidienne"):
            return True, date_max, Id

        content_a = soup.findAll("div", {"class": "content"})
        content = ''
        for x in content_a:
            for c in x.findAll('p'):
                content += c.text + "\n"
        
        if content == '':
            return True, date_max, Id

        date_author = soup.find("div",{"class":"ar-meta"}).findAll("span")
        author = date_author[0].text.strip()
        date_publication = dateparser.parse(date_author[1].text.lower().strip())

        language='Fr'
        country='Morocco'
        publication='Hespress FR'


        if(date_publication <= date_max):
            return False, date_publication, Id 
        Id = Id+1

        #sentiment analysis
        en_blob = TextBlob(str(content))
        sent = en_blob.sentiment.polarity
        if sent <-0.05 :
            sentiment = "Negative"
        elif sent > -0.05 and sent < 0.05 :
            sentiment = "Neutre"
        else :
            sentiment = "Positive"

        #Summarinzing
        tokens = tokenizer(content)
        sents = sent_tokenizer(content)
        word_counts = count_words(tokens)
        freq_dist = word_freq_distribution(word_counts)
        sent_scores = score_sentences(sents, freq_dist)
        summary, summary_sent_scores = summarize(sent_scores, 3) 

        writer.writerow({'Id': Id,'Title': title,'Author': author,'Date':date_publication,'Publication': publication,'URL': site,'Content': content,'Language': language,'Country': country, 'sentiment':sentiment,'Summary':summary})
        return True, date_publication, Id
    except :
        return True, date_max, Id


def add_newArticles_hespressFR(Id, date_max):
    p = 1
    stay = True
    fieldnames=['Id','Title','Author','Date', 'Publication','URL','Content','Language','Country','sentiment','Summary']
    with open('Articles.csv','a+',encoding="utf-8") as csv_file:
        article_writer = csv.writer(csv_file)
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        lastDate= date_max
        while stay: 
            site= "https://fr.hespress.com/tous-les-articles/page/"+str(p)
            p = p+1
            hdr = {'User-Agent': 'Mozilla/5.0'}
            req = Request(site,headers=hdr)
            page = urlopen(req)
            soup = BeautifulSoup(page)
            
            
            links = soup.findAll("a")
            i=0
            for link in links:
                sLink = link["href"].split("/")
                if len(sLink)>=4 and  str.isdigit(sLink[3].split("-")[0]):
                    stay, date, Id = scrap_article_HespressFR(link["href"], date_max, Id, writer)
                    if date > lastDate:
                        lastDate = date
                    if not stay:
                        break
                    i = i+1
            if i == 0:
                break
    return Id,lastDate

def update_hespressFR():
    fieldnames=['country', 'Publication', 'maxId', 'lastDate']
    with open('presse_lastUpdate.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        readData = [row for row in reader]
        found = False
        id_max = 0 
        lastdate = datetime(1 ,1 ,1 , 1, 1)
       
        for row in readData:
            if id_max < int(row['maxId']):
                id_max = int(row['maxId'])
            if row['country'] == "Morocco" and row['Publication'] == "Hespress FR" :
                lastdate = pd.to_datetime(row['lastDate'])  
                found = True
        id_max, lastdate = add_newArticles_hespressFR(id_max, lastdate)


    if found :
        with open('presse_lastUpdate.csv', "w", newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for row in readData:
                if row['country'] == "Morocco" and row['Publication'] == "Hespress FR" :
                    row['maxId'] = id_max
                    row['lastDate'] = lastdate
                    break
            writer.writerows(readData)
    if not found:
        with open('presse_lastUpdate.csv', "a+", newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow({'country' :'Morocco', 'Publication' : 'Hespress FR',  'maxId' : id_max, 'lastDate' :lastdate})

In [ ]:
import threading 
  
def hespressFR(): 
    update_hespressFR()
  
def leMatin(): 
    update_LeMatin()
  
if __name__ == "__main__": 
    # creating thread 
    t1 = threading.Thread(target=hespressFR) 
    t2 = threading.Thread(target=leMatin) 
    
    t1.start()
    t2.start() 

    t1.join()
    t2.join()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-e2e89b82ab82>", line 7, in leMatin
    update_LeMatin()
  File "<ipython-input-10-13d8a0f196b0>", line 104, in update_LeMatin
    id_max, lastdate = add_newArticles_LeMatin(id_max, lastdate)
  File "<ipython-input-10-13d8a0f196b0>", line 78, in add_newArticles_LeMatin
    stay, date, Id = scrap_article_LeMatin("https://lematin.ma"+link["href"],date_max, Id, writer)
  File "<ipython-input-10-13d8a0f196b0>", line 5, in scrap_article_LeMatin
    page = urlopen(req)
  File "/usr/lib/python3.6/urllib/request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/usr/lib/python3.6/urllib/reque